In [26]:
import json
import pandas as pd
import requests as req
import sqlite3
from datetime import date,datetime,timedelta
import numpy as np
import multiprocessing
import concurrent.futures


txt = req.get('https://health.data.ny.gov/api/views/xdss-u53e/rows.json?accessType=DOWNLOAD')
json_data = txt.json()

columns =[]
datatype=[]
for element in json_data['meta']['view']['columns']:
    columns.append(element['name'])
    datatype.append(element['dataTypeName'])

data = pd.DataFrame(json_data['data'],columns = columns)




def replace_special(str):
    str = str.replace('.','_')
    str = str.replace(' ','_')
    return(str)

def create_table(table_name):
    sql_create =f"CREATE TABLE {table_name}  (test_date DATE PRIMARY KEY NOT NULL, new_positives int NULL, cum_nbr_positives INT NULL,nbr_tests_performed INT NULL, cum_nbr_tests_performed INT NULL, load_date date );"
    conn.executescript(sql_create)
    return(f"{table_name} Table created successfully")

def insert_data(a,b,c,d,e,f):
    today_time = date.today()  
    sql_insert =f"INSERT INTO {a} (test_date, new_positives, cum_nbr_positives,nbr_tests_performed, cum_nbr_tests_performed, load_date)    VALUES ('{b}',{c} , {d}, {e},{f},'{today_time}' )"
    conn.execute(sql_insert)

def drop_table(table_name):
    sql_delete_all = f'delete from {table_name}'
    sql_drop = f"drop table {table_name}"
    conn.execute(sql_delete_all) 
    conn.execute(sql_drop)    
    
    
try:
    conn = sqlite3.connect('covid_virus_data.db')
except Error as e:
        print(e)

#get unique list of county names from the data to see if any new countes were added to the data                    
county_name = list(data['County'].str.lower().unique())        
county_name = list(map(replace_special,county_name))


new_sql="select name from sqlite_master where type = 'table'"

cursor1 = conn.execute(new_sql)
tables=[]
for row in cursor1:
    tables.append(row[0])


new_tables =list(set(county_name)-set(tables))

if len(new_tables)>0:
    for table in new_tables:
        create_table(table)

     

In [27]:

new_data = data[['County','Test Date','New Positives','Cumulative Number of Positives','Total Number of Tests Performed','Cumulative Number of Tests Performed']]

#craete a new column called Table_name to identify which table the specifc row would be inserted into
new_data['table_name']=new_data.apply(lambda row:replace_special(row['County']).lower(),axis=1)

#as infer_dtypes couldn't figure out all the datatypes. Enforce the datatype of each column in the DF
new_data.astype({'County':'str', 'New Positives': 'int32', 'Cumulative Number of Positives': 'int32','Total Number of Tests Performed':'int32', 'Cumulative Number of Tests Performed':'int32'}).dtypes
new_data['Test Date'] = pd.to_datetime(new_data['Test Date'])

#calculate yesterday's date. this is useful for incremental loads
yesterday = date.today() - timedelta(1)

#identify records from main dataframe for incremental loads
new_data_yesterday = new_data[~new_data['table_name'].isin(new_tables)]
new_data_yesterday=new_data_yesterday.loc[new_data_yesterday['Test Date']==str(yesterday)]

#idenify records for new counties or for first time loads
new_county_data = new_data[new_data['table_name'].isin(new_tables)]

C:\Users\sharanya\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\sharanya\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [28]:
#full data load for first time or if new counties are added
num_processes = multiprocessing.cpu_count()
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    pool.map(insert_data,new_county_data['table_name'],new_county_data['Test Date'],new_county_data['New Positives'],
             new_county_data['Cumulative Number of Positives'],new_county_data['Total Number of Tests Performed'],new_county_data['Cumulative Number of Tests Performed'])
    
# for index, row in new_county_data.iterrows():
#     insert_data(row)
conn.commit()

#fincremental load with yesterday's data
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    pool.map(insert_data,new_data_yesterday['table_name'],new_data_yesterday['Test Date'],new_data_yesterday['New Positives'],
             new_data_yesterday['Cumulative Number of Positives'],new_data_yesterday['Total Number of Tests Performed'],new_data_yesterday['Cumulative Number of Tests Performed'])


# for index, row in new_data_yesterday.iterrows():
#     insert_data(row)
# conn.commit()


#### Testing

In [30]:
abc = conn.execute("select * from jefferson")
for row in abc:
    print(row)

In [7]:
for i in tables:
    drop_table(i)
conn.commit()

In [11]:
print(new_tables)

['jefferson', 'schoharie', 'lewis', 'richmond', 'greene', 'washington', 'rensselaer', 'clinton', 'new_york', 'columbia', 'putnam', 'ontario', 'erie', 'chenango', 'ulster', 'schuyler', 'yates', 'westchester', 'montgomery', 'orleans', 'saratoga', 'livingston', 'rockland', 'suffolk', 'tompkins', 'seneca', 'chautauqua', 'chemung', 'niagara', 'albany', 'cayuga', 'st__lawrence', 'madison', 'franklin', 'delaware', 'allegany', 'steuben', 'oneida', 'essex', 'schenectady', 'tioga', 'orange', 'queens', 'cortland', 'sullivan', 'cattaraugus', 'hamilton', 'genesee', 'onondaga', 'warren', 'wayne', 'monroe', 'broome', 'fulton', 'dutchess', 'kings', 'otsego', 'nassau', 'wyoming', 'bronx', 'herkimer', 'oswego']


In [15]:
len(new_data_yesterday)

0

In [19]:
new_data_yesterday.head()

,County,Test Date,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed,table_name
356,Albany,2021-02-20,56,20392,3727,492881,albany
713,Allegany,2021-02-20,7,2839,1989,79998,allegany
1070,Bronx,2021-02-20,650,132881,10965,2376255,bronx
1427,Broome,2021-02-20,83,14228,3671,376050,broome
1784,Cattaraugus,2021-02-20,12,4281,331,108971,cattaraugus
